# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (linear problem), with wrong boundary markers

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla w \cdot \nabla v + \int_\Gamma \lambda v = \int_\Omega f v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example (accompanied by the corresponding DOLFINX one at the other link) assesses the robustness of the variable restriction to errors in the marking procedure.

In [ ]:
import matplotlib
import matplotlib.collections
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ufl import grad, inner, Measure
from dolfin import (assemble, DirichletBC, facets, Expression, Function, FunctionSpace, MeshFunction,
                    MeshView, MixedFunctionSpace, plot, solve, TestFunction, TestFunctions,
                    TrialFunction, TrialFunctions, UnitSquareMesh, vertices)

### Tolerance for boundary marking

In [ ]:
tol = np.finfo(float).eps
# tol = 0.4

### Auxiliary functions for plotting (adapted from tutorial 07)

In [ ]:
def plot_mesh_function(mesh_function):
    ax = plt.gca()
    ax.set_aspect("equal")
    mesh = mesh_function.mesh()
    tdim = mesh.topology().dim()
    points = mesh.coordinates()
    colors = ["b", "r"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap_bounds = [0, 0.5, 1]
    norm = matplotlib.colors.BoundaryNorm(cmap_bounds, cmap.N)
    assert mesh_function.dim() == tdim - 1
    linestyles = ["solid", "solid"]
    lines = list()
    lines_colors_as_int = list()
    lines_colors_as_str = list()
    lines_linestyles = list()
    for f in facets(mesh):
        mesh_function_f = mesh_function[f]
        vertices_ = [v.index() for v in vertices(f)]
        lines.append(points[vertices_][:, :2])
        lines_colors_as_int.append(mesh_function_f)
        lines_colors_as_str.append(colors[mesh_function_f])
        lines_linestyles.append(linestyles[mesh_function_f])
    mappable = matplotlib.collections.LineCollection(lines, cmap=cmap, norm=norm,
                                                     colors=lines_colors_as_str,
                                                     linestyles=lines_linestyles)
    mappable.set_array(np.array(lines_colors_as_int))
    ax.add_collection(mappable)
    ax.autoscale()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(mappable, cax=cax, cmap=cmap, norm=norm, boundaries=cmap_bounds, ticks=cmap_bounds)
    return ax

### Helper function for (possibly wrong!) boundary marking

In [ ]:
def mark_boundary(tol):
    def on_boundary(x):
        def near(x, a, tol):
            return np.abs(x - a) < tol

        return (near(x[0], 0.0, tol) or near(x[0], 1.0, tol)
                or near(x[1], 0.0, tol) or near(x[1], 1.0, tol))

    boundaries = MeshFunction("size_t", mesh, 1, 0)
    for f in facets(mesh):
        boundaries[f] = on_boundary(f.midpoint())

    return boundaries

### Mesh

In [ ]:
mesh = UnitSquareMesh(32, 32)
boundaries = mark_boundary(tol)

In [ ]:
plot(mesh)

In [ ]:
plot_mesh_function(boundaries)

In [ ]:
# Define submesh
submesh = MeshView.create(boundaries, 1)

In [ ]:
# Define associated measures
dx = Measure("dx")(domain=mesh)
ds = Measure("dx")(domain=submesh)

### Weak (and possibly wrong!) imposition of Dirichlet BCs

In [ ]:
# Define a mixed function space
V = FunctionSpace(mesh, "Lagrange", 2)
M = FunctionSpace(submesh, "Lagrange", 2)
W = MixedFunctionSpace(V, M)

In [ ]:
# Define trial and test functions
(u, l) = TrialFunctions(W)
(v, m) = TestFunctions(W)

In [ ]:
# Define problem block forms
g = Expression("sin(3 * pi * x[0] + 1) * sin(3 * pi * x[1] + 1)", degree=2)
a = inner(grad(u), grad(v)) * dx + l * v * ds + u * m * ds
f = v * dx + g * m * ds

In [ ]:
# Solve
ul = Function(W)
solve(a == f, ul, [], solver_parameters={"linear_solver": "direct"})

In [ ]:
# Split the block solution in components
(u, l) = (ul.sub(0), ul.sub(1))

In [ ]:
plot(u)

### Strong (and correct) imposition of Dirichlet BCs for comparison

In [ ]:
# Re-define trial and test functions
u_ex = TrialFunction(V)
v_ex = TestFunction(V)

In [ ]:
# Re-define forms
a_ex = inner(grad(u_ex), grad(v_ex)) * dx
f_ex = v_ex * dx

In [ ]:
# Define Dirichlet BC object on Gamma
boundaries_ex = mark_boundary(np.finfo(float).eps)
bc_ex = DirichletBC(V, g, boundaries_ex, 1)

In [ ]:
# Solve
u_ex = Function(V)
solve(a_ex == f_ex, u_ex, bc_ex, solver_parameters={"linear_solver": "direct"})

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex_norm = np.sqrt(assemble(inner(grad(u_ex), grad(u_ex)) * dx))
err_norm = np.sqrt(assemble(inner(grad(u_ex - u), grad(u_ex - u)) * dx))
print("Relative error is equal to", err_norm / u_ex_norm)